# Titanic Tutorial Copy2

## previously....

in the previous notebook, I followed two tutorial written in korean
- (titanic tutorial1)https://kaggle-kr.tistory.com/17?category=868316
- (titanic tutorial2)https://kaggle-kr.tistory.com/18?category=868316
Including part 1 and some of tutorial 2. I've prepared dataset for machine learning.
But it is not completed.... I'll continue to copy from 4.4. one-hot encoding.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno

%matplotlib inline
#Initiallize libraries and matplotlib.pyplot style like previous notebook (titanic_tutorial_copy1)

#let's check out dataframe before we start....
df_train=pd.read_csv('../input/titanic-tutorical-copy1to2/df_train1.csv')
df_test=pd.read_csv('../input/titanic-tutorical-copy1to2/df_test1.csv')

print(df_train)
print(df_test)

They have corresponding 7 columns and train set has 'survived' column, as I expected.  Unfortunately, there is NaN value found. Why does it happened? 



In [ ]:
#df_train.describe()
df_train.describe()
df_test.describe()
#df_test_origin=pd.read_csv('../input/titanic/test.csv')
#df_test_origin.describe()

It looks like I forgot to fill up some test.csv columns..... as 'age' feature is filled up with average of 'initial' feature, I can fill up again as average does not changed. Also 'fare' can be filled up as the average of pclass.

In [ ]:

for i in range(418):
    if df_test.Age_cat.isnull().values[i]==True:
        print('age is null ',i, df_test['Initial'][i])
    if df_test.Initial.isnull().values[i]==True:
        print('Initial is null ',i,df_test['Sex'][i],df_test['Age_cat'][i])
df_test[['Initial','Age_cat']].groupby(['Initial'], as_index=True).mean()
print(df_test.groupby('Initial').mean())


In [ ]:
df_test['Age_cat'][96]=3.0
df_test['Initial'][414]=3.0 #chosen 3.0 from sex and age
df_test.describe()

## One-hot encoding Embarked and Initial.

I'll not discuss One-hot encoding works. Anyway it can be easily done with get_dummies of pandas.

In [ ]:
df_train=pd.get_dummies(df_train, columns=['Embarked'],prefix='Embarked')
df_test=pd.get_dummies(df_test, columns=['Embarked'], prefix = 'Embarked')

In [ ]:
df_train = pd.get_dummies(df_train, columns=['Initial'], prefix='Initial')
df_test = pd.get_dummies(df_test, columns=['Initial'], prefix='Initial')
df_train.describe()

final dataset check-up before building machine learning model

In [ ]:
print(df_train.describe())
print(df_test.describe())

# Building machine learning model and prediction using the trained model

In [ ]:
from sklearn.ensemble import RandomForestClassifier #randomforest모델을 쓴답니다.
from sklearn import metrics #제출 전 사전평가를 위해 사용한다고 합니다.
from sklearn.model_selection import train_test_split #역시 사전평가에 쓰일 데이터를 준비하는 것 같습니다.

In [ ]:
X_train = df_train.drop('Survived', axis=1).values
target_label = df_train['Survived'].values
X_test = df_test.values
#결과를 알고 있는 train셋에서 임의로 일부를 떼어내 테스트 셋으로 사용, 모델의 성능을 예측해봅니다.

In [ ]:
X_tr, X_vld, y_tr, y_vld = train_test_split(X_train, target_label, test_size=0.3, random_state=2018)
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
prediction = model.predict(X_vld)

In [ ]:
print('{}명 중 {}%의 생존을 맞춤.'.format(y_vld.shape[0],100*metrics.accuracy_score(prediction,y_vld)))

## 4.3. Feature importance

어떤 feature가 생존률에 큰 영향을 줬는지 따져보는 것 같습니다.

## 4.4 Prediction on Test set

In [ ]:
from pandas import Series

feature_importance = model.feature_importances_
Series_feat_imp = Series(feature_importance, index=df_test.columns)

plt.figure(figsize=(8,8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

## 4.4 Prediction on Test set

In [ ]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
submission.head()

In [ ]:
prediction = model.predict(X_test)
submission['Survived']=prediction
submission.to_csv('./my_first_submission.csv',index=False)

submission 파일을 만들 수 있었습니다.